<p style="text-align: center;font-size:50px;">M. Ammar Siddique <br> BSDSF22M041</p>


<p style="text-align: center;font-size:50px;">Data Mining and Machine Learning </p>


<p style="text-align: center;font-size:50px;">Assignment # 4 </p>

                                                         
<p style="text-align: center;font-size:40px;">Crop Yield Data Set </p>

## Data Preprocessing

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("yield_data.csv")

# Handling missing values (if any)
df.dropna(inplace=True)

# Convert 'Year' to numeric (ensure it's treated as a continuous variable)
df['Year'] = df['Year'].astype(int)

# Frequency Encoding for categorical columns
for col in ['Area', 'Item']:
    df[col] = df[col].map(df[col].value_counts())

# Define independent (X) and dependent (y) variables
X = df.drop(columns=['hg/ha_yield'])  # Features
y = df['hg/ha_yield']                # Target Variable

# Apply Polynomial Transformation (Degree=2, Interaction Only)
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_poly = poly.fit_transform(X)

# Standardize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_poly)

# Splitting into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
import pandas as pd

# Convert X_poly back to DataFrame with proper column names
X_poly_df = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(X.columns))


# Print final shape of transformed dataset
print(f"Final number of features after encoding & polynomial transformation: {X_train.shape[1]}")



Final number of features after encoding & polynomial transformation: 21


In [3]:
df.head(25)



Area  Item  Year  hg/ha_yield  average_rain_fall_mm_per_year  \
0     99  4121  1990        36613                           1485   
1     99  4276  1990        66667                           1485   
2     99  3388  1990        23333                           1485   
3     99  3039  1990        12500                           1485   
4     99  3223  1990         7000                           1485   
5     99  3857  1990        30197                           1485   
6     99  4121  1991        29068                           1485   
7     99  4276  1991        77818                           1485   
8     99  3388  1991        28538                           1485   
9     99  3039  1991         6667                           1485   
10    99  3223  1991         6066                           1485   
11    99  3857  1991        20698                           1485   
12    99  4121  1992        24876                           1485   
13    99  4276  1992        82920                           1485   
14    99  3388  1992        40000                           1485   
15    99  3039  1992         3747                           1485   
16    99  3223  1992         4507                           1485   
17    99  3857  1992        24388                           1485   
18    99  4121  1993        24185                           1485   
19    99  4276  1993        98446                           1485   
20    99  3388  1993        41786                           1485   
21    99  3223  1993         7998                           1485   
22    99  3857  1993        29976                           1485   
23    99  4121  1994        25848                           1485   
24    99  4276  1994        81404                           1485   

    pesticides_tonnes  avg_temp  
0               121.0     16.37  
1               121.0     16.37  
2               121.0     16.37  
3               121.0     16.37  
4               121.0     16.37  
5               121.0     16.37  
6               121.0     15.36  
7               121.0     15.36  
8               121.0     15.36  
9               121.0     15.36  
10              121.0     15.36  
11              121.0     15.36  
12              121.0     16.06  
13              121.0     16.06  
14              121.0     16.06  
15              121.0     16.06  
16              121.0     16.06  
17              121.0     16.06  
18              121.0     16.05  
19              121.0     16.05  
20              121.0     16.05  
21              121.0     16.05  
22              121.0     16.05  
23              201.0     16.96  
24              201.0     16.96

# Forward selection

In [4]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression

def forward_selection(X, y, significance_level=0.05):
    selected_features = []
    remaining_features = list(X.columns)
    best_score = 0
    final_model = None

    while remaining_features:
        scores_with_candidates = []
        
        for feature in remaining_features:
            model_features = selected_features + [feature]
            X_train = sm.add_constant(X[model_features])  # Add intercept
            model = sm.OLS(y, X_train).fit()
            score = model.rsquared_adj  # Adjusted R²

            scores_with_candidates.append((score, feature, model))
        
        scores_with_candidates.sort(reverse=True, key=lambda x: x[0])  # Sort by best Adjusted R²
        
        best_new_score, best_feature, best_model = scores_with_candidates[0]

        if best_new_score > best_score:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            best_score = best_new_score
            final_model = best_model
        else:
            break  # Stop if no improvement

    return selected_features, final_model

# Apply Forward Selection
import pandas as pd

# Convert X_poly back to DataFrame with proper column names
X_poly_df = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(X.columns))

# Apply Forward Selection with the corrected DataFrame
selected_features, forward_model = forward_selection(X_poly_df, y)


# Print Summary
print("Selected Features:", selected_features)
print(forward_model.summary())


Selected Features: ['Item avg_temp', 'Year', 'Area Year', 'Item Year', 'Year avg_temp', 'Area Item', 'Area avg_temp', 'average_rain_fall_mm_per_year', 'average_rain_fall_mm_per_year avg_temp', 'Item average_rain_fall_mm_per_year', 'Area average_rain_fall_mm_per_year', 'Item pesticides_tonnes', 'Area pesticides_tonnes', 'average_rain_fall_mm_per_year pesticides_tonnes', 'pesticides_tonnes', 'pesticides_tonnes avg_temp', 'Area', 'Year pesticides_tonnes']
                            OLS Regression Results                            
Dep. Variable:            hg/ha_yield   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     223.4
Date:                Sat, 08 Mar 2025   Prob (F-statistic):               0.00
Time:                        02:48:32   Log-Likelihood:            -3.5874e+05
No. Observations:               28242   AIC:                         

# Backward elimination

In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

def backward_elimination(X, y, significance_level=0.05):
    X = pd.DataFrame(X)  # Ensure X is a DataFrame
    X = sm.add_constant(X)  # Add intercept term (constant)
    
    num_vars = X.shape[1]
    for i in range(num_vars):
        model = sm.OLS(y, X).fit()  # Fit OLS model
        p_values = model.pvalues  # Get p-values
        
        max_p_value = p_values.max()  # Find the max p-value
        if max_p_value > significance_level:  
            feature_to_remove = p_values.idxmax()  # Feature with max p-value
            X = X.drop(columns=[feature_to_remove])  # Remove feature
        else:
            break  # Stop if all p-values are below threshold
    
    final_model = sm.OLS(y, X).fit()  # Final model after elimination
    return X.columns, final_model

# Convert X_poly to DataFrame (if needed)
X_poly_df = pd.DataFrame(X_poly)

# Apply Backward Elimination
selected_features, backward_model = backward_elimination(X_poly_df, y)

# Print Results
print("Selected Features:", selected_features)
print(backward_model.summary())  # Show final model summary


Selected Features: Index(['const', 0, 1, 2, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 18, 19, 20], dtype='object')
                            OLS Regression Results                            
Dep. Variable:            hg/ha_yield   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     236.6
Date:                Sat, 08 Mar 2025   Prob (F-statistic):               0.00
Time:                        02:48:41   Log-Likelihood:            -3.5874e+05
No. Observations:               28242   AIC:                         7.175e+05
Df Residuals:                   28224   BIC:                         7.177e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------

# Bidirectional Elimination

In [6]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

def bidirectional_elimination(X, y, significance_level=0.05):
    X = pd.DataFrame(X)  # Ensure X is a DataFrame
    X = sm.add_constant(X)  # Add intercept
    selected_features = list(X.columns)  # Start with all features
    removed_features = []
    
    while True:
        model = sm.OLS(y, X[selected_features]).fit()
        p_values = model.pvalues
        
        # **Backward Step**: Remove feature with highest p-value above threshold
        max_p = p_values.max()
        if max_p > significance_level:
            feature_to_remove = p_values.idxmax()
            selected_features.remove(feature_to_remove)
            removed_features.append(feature_to_remove)
            continue  # Re-evaluate after removal
        
        # **Forward Step**: Try adding previously removed features
        added = False
        for feature in removed_features[:]:  # Iterate over a copy of the list
            test_features = selected_features + [feature]
            model = sm.OLS(y, X[test_features]).fit()
            if model.pvalues[feature] < significance_level:  # If now significant, add back
                selected_features.append(feature)
                removed_features.remove(feature)
                added = True
        
        if not added:  # Stop if no changes in forward step
            break
    
    final_model = sm.OLS(y, X[selected_features]).fit()
    return selected_features, final_model

# Convert X_poly to DataFrame (if needed)
X_poly_df = pd.DataFrame(X_poly)

# Apply Bidirectional Elimination
selected_features, stepwise_model = bidirectional_elimination(X_poly_df, y)

# Print Results
print("Selected Features:", selected_features)
print(stepwise_model.summary())  # Show final model summary

Selected Features: ['const', 0, 1, 2, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 18, 19, 20]
                            OLS Regression Results                            
Dep. Variable:            hg/ha_yield   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     236.6
Date:                Sat, 08 Mar 2025   Prob (F-statistic):               0.00
Time:                        02:48:47   Log-Likelihood:            -3.5874e+05
No. Observations:               28242   AIC:                         7.175e+05
Df Residuals:                   28224   BIC:                         7.177e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------

# All Columns

In [7]:
import statsmodels.api as sm

# Add intercept term (constant) for OLS regression
X_poly_with_const = sm.add_constant(X_poly)  

# Train the model using OLS (Ordinary Least Squares)
full_model = sm.OLS(y, X_poly_with_const).fit()

# Print Model Summary
print(full_model.summary())


                            OLS Regression Results                            
Dep. Variable:            hg/ha_yield   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     191.6
Date:                Sat, 08 Mar 2025   Prob (F-statistic):               0.00
Time:                        02:48:51   Log-Likelihood:            -3.5873e+05
No. Observations:               28242   AIC:                         7.175e+05
Df Residuals:                   28220   BIC:                         7.177e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.151e+06   8.51e+05     -2.528      0.0